# 0. Import

In [1]:
import pickle               as pkl
import pandas               as pd
import numpy                as np
import plotly_express       as px
import matplotlib.pyplot    as plt

from IPython.display         import HTML

from sklearn                 import datasets            as ds
from sklearn                 import tree                as tr
from sklearn                 import metrics             as mt
from sklearn                 import preprocessing       as pp
from sklearn                 import model_selection     as ms
from sklearn.preprocessing   import RobustScaler, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate
from sklearn.ensemble        import RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier
from xgboost                 import XGBClassifier
from sklearn.linear_model    import LogisticRegression, SGDClassifier
from sklearn.neighbors       import KNeighborsClassifier
from sklearn.tree            import DecisionTreeClassifier
from sklearn.ensemble        import RandomForestClassifier, BaggingClassifier
from sklearn.metrics         import accuracy_score, recall_score, precision_score, balanced_accuracy_score, f1_score, roc_curve, confusion_matrix
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.naive_bayes     import GaussianNB
from lightgbm                import LGBMClassifier

'\nfrom sklearn                 import preprocessing       as pp\nfrom sklearn                 import model_selection     as ms\nfrom sklearn.preprocessing   import RobustScaler, MinMaxScaler, StandardScaler\nfrom sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate\nfrom sklearn.ensemble        import RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier\nfrom xgboost                 import XGBClassifier\nfrom sklearn.linear_model    import LogisticRegression, SGDClassifier\nfrom sklearn.neighbors       import KNeighborsClassifier\nfrom sklearn.tree            import DecisionTreeClassifier\nfrom sklearn.ensemble        import RandomForestClassifier, BaggingClassifier\nfrom sklearn.metrics         import accuracy_score, recall_score, precision_score, balanced_accuracy_score, f1_score, roc_curve, confusion_matrix\nfrom sklearn.model_selection import GridSearchCV, RandomizedSearchCV\nfrom sklearn.naive_bayes     import GaussianNB\nfrom lightgbm

## 0.1 Load Dataset

In [ ]:
x_train = pickle.load(open('../dataset/processed/x_train.pkl', 'rb'))
y_train = pickle.load(open('../dataset/processed/y_train.pkl', 'rb'))
x_val = pickle.load(open('../dataset/processed/x_val.pkl', 'rb'))
y_val = pickle.load(open('../dataset/processed/y_val.pkl', 'rb'))
x_teste = pickle.load(open('../dataset/processed/x_test.pkl', 'rb'))
y_teste = pickle.load(open('../dataset/processed/y_test.pkl', 'rb'))

# Algoritmos de Treinamento

In [ ]:
def MachineLearning(dataset):
    SEED = 42

    lista_de_medidas = ['accuracy', 'recall', 'precision', 'balanced_accuracy', 'f1']
    nome_das_medidas = ['acurácia', 'recall', 'precision', 'eficiência', 'f1-score']

    lista_de_modelos = [XGBClassifier(learning_rate=0.1, n_estimators=100, max_depth=3),
                        SGDClassifier(loss='log_loss', random_state=SEED, n_jobs=-1), 
                        LGBMClassifier(random_state=SEED, n_jobs=-1, force_row_wise=True, ),
                        GaussianNB(),
                        LogisticRegression(max_iter=220, random_state=SEED),
                        DecisionTreeClassifier(random_state=SEED),
                        KNeighborsClassifier(n_neighbors=5,  weights='distance',n_jobs=-1),
                        BaggingClassifier(),
                        RandomForestClassifier(random_state=SEED)]

    nome_dos_modelos = ['XGBClassifier', 
                        'SGDClassifier', 
                        'LGBMClassifier', 
                        'GaussianNB',
                        'Regressão Logística',
                        'DecisionTreeClassifier',
                        'KNN',
                        'BaggingClassifier',
                        'RandomForestClassifier']

    resultados0 = {}

    for i in range(len(lista_de_modelos)):
        print('Rodando modelo: ' + nome_dos_modelos[i])
        accs_vc = cross_validate(lista_de_modelos[i], x_train, y_train, scoring = lista_de_medidas)
        # cv = kf,


        acc = accs_vc['test_accuracy'].mean()
        sen = accs_vc['test_recall'].mean()
        vpp = accs_vc['test_precision'].mean()
        bac = accs_vc['test_balanced_accuracy'].mean()
        f1s = accs_vc['test_f1'].mean()

        resultados0[nome_dos_modelos[i]] = [acc, sen, vpp, f1s, bac]
    
    resultados = pd.DataFrame(resultados0, index = nome_das_medidas).T

    return resultados

In [ ]:
MachineLearning(df_train)

In [ ]:
def train_and_predict(model, x_train, y_train, x_teste):
    model.fit(x_train, y_train)
    y_pred = model.predict(x_teste)
    
    accuracy = accuracy_score(y_teste, y_pred)
    recall = recall_score(y_teste, y_pred)
    precision = precision_score(y_teste, y_pred)
    balanced_accuracy = balanced_accuracy_score(y_teste, y_pred)
    f1 = f1_score(y_teste, y_pred)

    print(model)
    print("Accuracy:", accuracy)
    print("Recall:", recall)
    print("Precision:", precision)
    print("Balanced Accuracy:", balanced_accuracy)
    print("F1 Score:", f1)

In [ ]:
xgb = XGBClassifier(learning_rate=0.1, n_estimators=100, max_depth=3)
predic_xgb = train_and_predict(xgb, x_train, y_train, x_teste)

predic_xgb

In [ ]:
# Modelo SGDClassifier
SEED = 42
lgbm = LGBMClassifier(random_state=SEED, n_jobs=-1, force_row_wise=True)
predic_lgbm = train_and_predict(lgbm, x_train, y_train, x_teste)


predic_lgbm

In [ ]:
# Modelo SGDClassifier
sgd = SGDClassifier(loss='log_loss', random_state=SEED, n_jobs=-1)
predic_sgd = train_and_predict(sgd, x_train, y_train, x_teste)


predic_sgd

In [ ]:
# Modelo SGDClassifier
lr = LogisticRegression(max_iter=220, random_state=SEED)
predic_lr = train_and_predict(lr, x_train, y_train, x_teste)

predic_lr

# Fine Tuning

In [ ]:
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 6, 9],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# Cria uma instância do GridSearchCV com o modelo XGBoost e os parâmetros definidos
grid_search = GridSearchCV(XGBClassifier(), param_grid=param_grid, scoring='f1', cv=5)
grid_search.fit(x_train, y_train)

# Imprime os melhores parâmetros encontrados
print(grid_search.best_params_)

In [ ]:
# # Modelo selecionado
# XGBoosting já aplica o boosting então não precisa dessa etapa.

# final_model = XGBClassifier( colsample_bytree= 0.8, learning_rate= 0.1, max_depth= 6, n_estimators= 100, subsample= 0.8 )
final_model = XGBClassifier( colsample_bytree= 1.0, learning_rate= 0.1, max_depth= 3, n_estimators= 100, subsample= 0.8 )


final_model.fit(x_train, y_train)
y_pred = final_model.predict( x_teste )

# Calculando as métricas de avaliação com base nas previsões da validação cruzada
accuracy = accuracy_score(y_teste, y_pred)
recall = recall_score(y_teste, y_pred)
precision = precision_score(y_teste, y_pred)
balanced_accuracy = balanced_accuracy_score(y_teste, y_pred)
f1 = f1_score(y_teste, y_pred)

print("Accuracy:", accuracy)
print("Recall:", recall)
print("Precision:", precision)
print("Balanced Accuracy:", balanced_accuracy)
print("F1 Score:", f1)

# Cross Val

In [ ]:
# Validação cruzada Leave One Out
kf = StratifiedKFold(n_splits = 10)

lista_de_medidas = ['precision', 'recall', 'f1']

modelo_final_cv = XGBClassifier( colsample_bytree= 1.0, learning_rate= 0.1, max_depth= 3, n_estimators= 100, subsample= 0.8 )  

kf_scores = cross_validate(modelo_final_cv, x_final, y_final, cv=kf, scoring=lista_de_medidas, n_jobs=-1)

for medida in lista_de_medidas:
    print(f"Average {medida}: {kf_scores['test_' + medida].mean():.4f} (+/- {kf_scores['test_' + medida].std() * 2:.4f})")

In [ ]:
threshold = 0.45

modelo_final_cv = XGBClassifier(learning_rate=0.1, n_estimators=100, max_depth=3)

# Fit the model
modelo_final_cv.fit(x_final, y_final)

# Predict the probabilities
modelo_final_cv_probs = modelo_final_cv.predict_proba(x_teste)

# Keep probabilities for the positive outcome only
probs = modelo_final_cv_probs[:, 1]

# Apply the threshold
y_pred = np.where(probs > threshold, 1, 0)

# Calculate the confusion matrix
cm = confusion_matrix(y_teste, y_pred)

# Create a heatmap of the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title("Confusion Matrix")
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.show()

# 4.0 Juntando os Dataset

## Treino (80%) -> Treino(60%) + Validação(20%)

In [ ]:
#Treino (80%) -> Treino(60%) + Validação(20%)
train_80 = np.concatenate( (x_train, x_val) )
val_80 = np.concatenate( (y_train, y_val) )

#Treino Last -> Modelo + Melhor Parêmetro + Treino (80%)
model_last = tr.DecisionTreeClassifier(max_depth=9)
model_last.fit(train_80, val_80 )

yhat_test = model_last.predict(x_teste)
acc_test = mt.accuracy_score(y_teste, yhat_test)

## Treino ( Treino(60%) + Validação(20%) ) + Teste(20%)

In [ ]:
#Treino ( Treino(60%) + Validação(20%) ) + Teste(20%)
train_100 = np.concatenate( (train_80, x_teste) )
val_100 = np.concatenate( (val_80, y_teste) )

# Modelo em Produção
model_last = tr.DecisionTreeClassifier(max_depth=9)
model_last.fit( train_100, val_100 )

# 4.0 Exportar para o Kaggle

In [3]:
data_set.to_csv('../dataset/production/data_set_kaggle.csv', index=False)